In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 100)

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Lasso, Ridge

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
pd.options.display.max_columns = None

In [ ]:
data = pd.read_csv('input/train.csv')

In [ ]:
data.head(10)

In [ ]:
data.shape

In [ ]:
data['Id'].nunique()

In [ ]:
data.info()

#### Healthcare_1

In [ ]:
data = data.drop('Healthcare_1', axis=1)

In [ ]:
data.shape

#### describe

In [ ]:
data.describe()

Пример использования медианы (4 друга - заработные платы в тыс. руб.)

In [ ]:
60, 80, 100, 5000

In [ ]:
(60 + 80 + 100 + 5000) / 4

In [ ]:
60, 80, 100, 5000

In [ ]:
(80 + 100) / 2

#### HouseYear

In [ ]:
2.005201e+07 == 2.005201 * 10 ** 7

In [ ]:
2.005201 * 10 ** 7

In [ ]:
data.loc[data['HouseYear'] > 2020, :]

In [ ]:
data.loc[data['HouseYear'] == 20052011, 'HouseYear'] = 2008

In [ ]:
data.loc[data['HouseYear'] == 4968, 'HouseYear'] = 1968

In [ ]:
data.loc[data['HouseYear'] < 1900, :]

In [ ]:
data['HouseYear'].describe()

#### Rooms

In [ ]:
data.Rooms.describe()

In [ ]:
data.loc[data['Rooms'] > 5, :]

In [ ]:
data.loc[data['Id'].isin([5927, 14003, 14865]), 'Rooms'] = 2

In [ ]:
data.loc[data['Id'] == 8491, 'Rooms'] = 1

#### Small Square, LifeSquare

In [ ]:
data.loc[data['Square'] < 15, :].shape

In [ ]:
small_square_ids = data.loc[data['Square'] < 15, 'Id'].values
small_square_ids

In [ ]:
data.loc[data['Id'].isin(small_square_ids), :]

In [ ]:
mean_square = data.groupby('Rooms', as_index=False)[['Square']].mean().rename(columns={'Square': 'mean_square'})
mean_square

In [ ]:
data = pd.merge(data, mean_square, on='Rooms', how='left')

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.head(10)

In [ ]:
data.loc[data['Id'].isin(small_square_ids), 'Square'] = data['mean_square']

In [ ]:
data.loc[data['Id'].isin(small_square_ids), :]

In [ ]:
data.loc[data['Id'].isin(small_square_ids), 'LifeSquare'] = data['Square']

In [ ]:
data.loc[data['Id'].isin(small_square_ids), :]

#### Large Square, LifeSquare

In [ ]:
large_square_ids = data.loc[data['Square'] > 150, 'Id'].values

In [ ]:
large_square_ids

In [ ]:
data.loc[data['Square'] > 150, :].head()

In [ ]:
data.loc[data['Id'].isin(large_square_ids), 'Square'] = data['mean_square']

In [ ]:
data.loc[data['Id'].isin(large_square_ids), 'LifeSquare'] = data['Square']

In [ ]:
data.loc[data['Id'].isin(large_square_ids), :].head()

#### Ecology_2

In [ ]:
data['Ecology_2'].unique()

In [ ]:
data['Ecology_2'].value_counts()

In [ ]:
data['Ecology_2'] = (data['Ecology_2'] == 'B').astype(int)

In [ ]:
data['Ecology_2'].value_counts(normalize=True)

#### fillna LifeSquare

In [ ]:
data['LifeSquare'].describe()

In [ ]:
data['Square'].describe()

In [ ]:
data['LifeSquare'] = data['LifeSquare'].fillna(data['Square'])

In [ ]:
data['LifeSquare'].describe()

In [ ]:
data.loc[data['LifeSquare'] > data['Square'], :].shape

In [ ]:
data.loc[data['LifeSquare'] > data['Square'], 'LifeSquare'] = data['Square']

In [ ]:
data['LifeSquare'].describe()

#### Floor

In [ ]:
plt.scatter(data['Floor'], data['Price']);

In [ ]:
data[data['Floor'] > 30].head()

#### HouseFloor

In [ ]:
plt.scatter(data['HouseFloor'], data['Price']);

In [ ]:
data[data['HouseFloor'] > 80].head()

In [ ]:
# mean_house_floor = data[data['HouseFloor'] > 80]
# data[data['HouseFloor'] > 80]['HouseFloor'] =\
#     data[(data['HouseFloor'] <= 80) & data['HouseYear']]['HouseFloor'].mean()

mean_house_floor = data.loc[(data['HouseFloor'] <= 80) & (data['HouseYear'] == 1977)]['HouseFloor'].mean()
data.loc[data['HouseFloor'] > 80, 'HouseFloor'] = mean_house_floor


In [ ]:
assert np.int(data[data.index == 6131]['HouseFloor']) <= 80

In [ ]:
data.min()

In [ ]:
len(data[(data['HouseYear'] == 1977) & (data['HouseFloor'] <= 1)])

In [ ]:
# feat = 'HouseFloor'
# year = 1977

# min_year = np.int(data['HouseYear'].min())
# max_year = np.int(data['HouseYear'].max())
# display((min_year, max_year))

# for year in np.sort(data['HouseYear'].unique()):
#     if year < 1964:
#         continue
#     data_year = data[data['HouseYear'] == year]
#     data_year_2 = data[(data['HouseYear'] == year) & (data['Floor'] > data['HouseFloor'])]
#     plt.scatter(data_year[feat], data_year['Price'], zorder=2);
#     plt.scatter(data_year_2[feat], data_year_2['Price'], zorder=2);
    
#     min_hf = np.int(data[data['HouseYear'] == year][feat].min())
#     max_hf = np.int(data[data['HouseYear'] == year][feat].max() + 0.99)
    
#     plt.xticks(range(min_hf, max_hf))
#     plt.gca().xaxis.grid(True)
#     plt.gcf().set_size_inches(11, 3)
#     plt.title(year)
#     plt.show()

In [ ]:
data.loc[data['Floor'] > data['HouseFloor'],
     ['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear']]

In [ ]:
def add_mean_column(feat):
    global data
    new_feat = 'MP_' + feat
    data_stat1 = data.groupby([feat, 'Rooms'], as_index=False)[['Price']]\
        .mean().rename(columns={'Price': new_feat})
    data = pd.merge(data, data_stat1, on=[feat, 'Rooms'], how='left')

In [ ]:
add_mean_column('DistrictId')
add_mean_column('Floor')
add_mean_column('HouseFloor')
add_mean_column('HouseYear')
add_mean_column('Social_3')
add_mean_column('Helthcare_2')
add_mean_column('Shops_1')

In [ ]:
data['FloorRev'] = data['HouseFloor'] - data['Floor']
add_mean_column('FloorRev')

In [ ]:
data['FloorError'] = (data['Floor'] > data['HouseFloor']).astype(int)

In [ ]:
data.head(10)

In [ ]:
data.hist(bins=40, figsize=(15,10), zorder=2)
plt.show()

In [ ]:
def print_unique():
    for feat in data.columns:
        nunique = data[feat].nunique()
        print(f'{feat:15} {nunique:5}', end='')
        unique = '[...]'
        if nunique <= 25:
            unique = data[feat].unique()
            unique.sort()
        print('    ', end='')
        print(unique)

In [ ]:
print_unique()

In [ ]:
def _ab_to_01(feat):
    try:
        data[feat] = data[feat].replace(['A', 'B'], [0, 1])
    except:
        pass

In [ ]:
_ab_to_01('Ecology_2')
_ab_to_01('Ecology_3')
_ab_to_01('Shops_2')

In [ ]:
print_unique()

In [ ]:
data.head(10)

In [ ]:
# data.loc[data['HouseFloor'] == 0]
mean_house_floor = data.loc[data['HouseFloor'] != 0]['HouseFloor'].mean()
data.loc[data['HouseFloor'] == 0] = mean_house_floor

In [ ]:
data['FloorK'] = data['Floor'] / data['HouseFloor']
data['FloorK'].describe()

In [ ]:
feats = [
#     'DistrictId',
    'Rooms',
    'Square',
    'LifeSquare',
    'KitchenSquare',
    'Floor',
    'HouseFloor',
#     'HouseYear',
    'Ecology_1',
    'Ecology_2',
    'Ecology_3',
    'Social_1',
    'Social_2',
    #Social_3',
#     'Helthcare_2',
    #Shops_1',
    'Shops_2',
    'MP_DistrictId',
    'MP_HouseFloor',
    'MP_HouseYear',
    'MP_Social_3',
    'MP_Helthcare_2',
    'MP_Shops_1',
#     'FloorK',
    'FloorRev',
#    'FloorError',
    ]

X = pd.DataFrame(data, columns=feats)

def _ctgr(feat):
    try:
        X[feat] = X[feat].astype('category')
    except:
        pass

# _ctgr('DistrictId')
# _ctgr('Rooms')
# _ctgr('Floor')
# _ctgr('FloorRev')
# _ctgr('HouseFloor')
# _ctgr('HouseYear')
# _ctgr('Ecology_2')
# _ctgr('Ecology_3')
_ctgr('Helthcare_2')
# _ctgr('Shops_1')
# _ctgr('Shops_2')

X = pd.get_dummies(X)
feats = X.columns
X.head()

In [ ]:
X.info()

In [ ]:
X.describe()

In [ ]:
y = pd.DataFrame(data, columns=['Price'])
y.head()

In [ ]:
y.info()

In [ ]:
y.describe()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=11)

In [ ]:
lr = LinearRegression()
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)

X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)
X_test_scaled

lr.fit(X_train_scaled, y_train)

In [ ]:
plt.rcParams.update({'font.size': 7})
plt.rcParams["axes.grid.axis"] = "y"
plt.rcParams["axes.grid"] = True

plt.barh(X_train.columns, lr.coef_.flatten(), zorder=2)
plt.xlabel('Вес признака')
plt.ylabel('Признак')

fig = plt.gcf()
fig.set_size_inches(11.2, 4)
plt.gca().invert_yaxis()

In [ ]:
def create_model(X_train, y_train, X_test, y_test, feats, model):
    print(1)
    model.fit(X_train.loc[:, feats], y_train)
    print(2)
    y_pred = model.predict(X_test.loc[:, feats])
    print(3)
    ret = mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)
    return ret

In [ ]:
create_model(X_train_scaled, y_train, X_test_scaled, y_test, feats, LinearRegression())